In [ ]:
# !pip install spotipy

In [1]:
import operator
import sys
import pyspark
from pyspark.sql import SparkSession, types
from pyspark import SparkConf, SparkContext
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [12]:
CLIENT_ID = 'd3e048fac0e8438cab168ac60459044c'
CLIENT_SECRET = '4d47925e9199481fb41cc0c1406de938'

In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = CLIENT_ID
secret = CLIENT_SECRET
client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# create lists of columns to be used when reading/merging the csv's
columns = ['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend', 'streams']
DATA_PATH = "gs://big_data_spotify_bucket/charts.csv"
# Spark settings
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
# sp.audio_features(track_url)[0]

In [5]:
# Load the data, cache this since we're accessing this each iteration
data = spark.read.format("csv").option("header", "true").load(DATA_PATH).cache()
#===============================

In [6]:
data = data.withColumn("rank", data["rank"].cast("int")).withColumn("streams", data["streams"].cast("int"))
data.printSchema()
data.show(20)
# data.count()

root
 |-- title: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: integer (nullable = true)



+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|
|              Safari|   4|2017-01-01|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|
|         Shaky Shaky|   5|2017-01-01|        Daddy Yankee|https://open.spot...|Argentina|top200|      MOVE_UP| 153956|
|         Traicionera|   6|2017-01-01|  

In [7]:

# data.take(1)
url_column = data.select('url')
print(url_column.take(1)[0][0])
sp.audio_features(url_column.take(1)[0][0])[0]
7w87IxuO7BDcJ3YUqCyMTT

https://open.spotify.com/track/6mICuAdrwEjh6Y6lroV2Kg


{'danceability': 0.852,
 'energy': 0.773,
 'key': 8,
 'loudness': -2.921,
 'mode': 0,
 'speechiness': 0.0776,
 'acousticness': 0.187,
 'instrumentalness': 3.05e-05,
 'liveness': 0.159,
 'valence': 0.907,
 'tempo': 102.034,
 'type': 'audio_features',
 'id': '6mICuAdrwEjh6Y6lroV2Kg',
 'uri': 'spotify:track:6mICuAdrwEjh6Y6lroV2Kg',
 'track_href': 'https://api.spotify.com/v1/tracks/6mICuAdrwEjh6Y6lroV2Kg',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6mICuAdrwEjh6Y6lroV2Kg',
 'duration_ms': 195840,
 'time_signature': 4}

In [8]:
data_pd = data.limit(100000).toPandas()
data_pd.shape

(100000, 9)

In [ ]:
track_features = []
tf_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
cols_to_drop2 = ['key', 'mode', 'type', 'track_href', 'analysis_url']
for index, row in data_pd.iterrows():
    if index > 30000:
        if index % 1000 == 0:
            print(index)
    #         print(sp.audio_features(row['url'])[0])  
            temp_df = tf_df.drop(columns=cols_to_drop2)
            new_df = pd.concat([data_pd, temp_df], axis=1)
    #         print(new_df.shape)
            new_df.to_csv("gs://big_data_spotify_bucket/processed_data_v2.csv")

        audio_features = sp.audio_features(row['url'])[0]
        tf_df = tf_df.append(audio_features, ignore_index = True)
    
#     track_features.append(audio_features)

# tf_df.head()
temp_df = tf_df.drop(columns=cols_to_drop2)
new_df = pd.concat([data_pd, temp_df], axis=1)
print(new_df.shape)
new_df.to_csv("gs://big_data_spotify_bucket/processed_data_v2.csv")

31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [ ]:
# cols_to_drop2 = ['key', 'mode', 'type', 'track_href', 'analysis_url']
# tf_df = tf_df.drop(columns=cols_to_drop2)
# print(track_df.info())
# print(tf_df.info())
# new_df = pd.concat([data_pd, tf_df], axis=1)
# new_df.head()
# sparkDF=spark.createDataFrame(new_df) 
# sparkDF.printSchema()
# sparkDF.show(20)

In [ ]:
# new_df.shape

In [ ]:
# save data
# new_df.to_csv("gs://big_data_spotify_bucket/processed_data.csv")

In [ ]:
# # Define the number of splits you want
# n_splits = 6
  
# # Calculate count of each dataframe rows
# each_len = 26173514 // n_splits
  
# # Create a copy of original dataframe
# copy_df = data
  
# # Iterate for each dataframe
# i = 0
# while i < n_splits:
  
#     # Get the top `each_len` number of rows
#     temp_df = copy_df.limit(each_len)
  
#     # Truncate the `copy_df` to remove
#     # the contents fetched for `temp_df`
#     copy_df = copy_df.subtract(temp_df)
  
#     # View the dataframe
# #     print(temp_df.count())
# #     temp_df.show(20)
#     temp = temp_df.toPandas()
#     print(temp.shape)
#     # Increment the split number
#     i += 1

In [14]:
sp.audio_features('7w87IxuO7BDcJ3YUqCyMTT')[0]

{'danceability': 0.733,
 'energy': 0.71,
 'key': 5,
 'loudness': -5.849,
 'mode': 0,
 'speechiness': 0.0292,
 'acousticness': 0.145,
 'instrumentalness': 0.115,
 'liveness': 0.0956,
 'valence': 0.965,
 'tempo': 127.975,
 'type': 'audio_features',
 'id': '7w87IxuO7BDcJ3YUqCyMTT',
 'uri': 'spotify:track:7w87IxuO7BDcJ3YUqCyMTT',
 'track_href': 'https://api.spotify.com/v1/tracks/7w87IxuO7BDcJ3YUqCyMTT',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7w87IxuO7BDcJ3YUqCyMTT',
 'duration_ms': 239600,
 'time_signature': 4}

In [45]:
data1 = pd.read_csv("gs://big_data_spotify_bucket/processed_data.csv")
data2 = pd.read_csv("gs://big_data_spotify_bucket/processed_data_v2.csv")

/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3194: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [47]:
# new_df = pd.concat([data1, data2], axis=0)
# # new_df.to_csv("gs://big_data_spotify_bucket/processed_data_v3.csv")
# new_df2 = new_df.iloc[0:100000]
# new_df2
# data1.drop(columns=data1.columns[0], axis=1, inplace=True)
# data2.drop(columns=data2.columns[0], axis=1, inplace=True)


title                   0
rank                    0
date                    0
artist                  0
url                     0
region                  0
chart                   0
trend                   0
streams             11142
danceability        70006
energy              70006
loudness            70006
speechiness         70006
acousticness        70006
instrumentalness    70006
liveness            70006
valence             70006
tempo               70006
id                  70006
uri                 70006
duration_ms         70006
time_signature      70006
dtype: int64

In [48]:
data1.isnull().sum()

title                   0
rank                    0
date                    0
artist                  0
url                     0
region                  0
chart                   0
trend                   0
streams             11142
danceability        70006
energy              70006
loudness            70006
speechiness         70006
acousticness        70006
instrumentalness    70006
liveness            70006
valence             70006
tempo               70006
id                  70006
uri                 70006
duration_ms         70006
time_signature      70006
dtype: int64

In [49]:
data2.isnull().sum()

title                   0
rank                    0
date                    0
artist                  0
url                     0
region                  0
chart                   0
trend                   0
streams             11142
danceability        30014
energy              30014
loudness            30014
speechiness         30014
acousticness        30014
instrumentalness    30014
liveness            30014
valence             30014
tempo               30014
id                  30014
uri                 30014
duration_ms         30014
time_signature      30014
dtype: int64